## Análisis del Dataset de Importación de Vehículos

**1. Descripción del dataset**
- Total de filas: _(se mostrará en ejecución)_
- Total de variables: _(se mostrará en ejecución)_

**2. Variables que necesitan limpieza**
- `pais_de_proveniencia`, `marca`, `modelo_del_vehiculo`, `linea`, `aduana_de_ingreso`, `distintivo`:
  - Presentan mayúsculas/minúsculas inconsistentes, espacios, posibles repeticiones o errores.

**3. Estrategia de limpieza**
- Convertir todo texto a mayúsculas (o capitalizar en casos como el combustible).
- Eliminar espacios innecesarios.
- Convertir fechas a tipo `datetime` para permitir análisis temporal.
- No se eliminarán errores ortográficos si provienen del origen, para conservar fidelidad en informes posteriores.
- No se eliminan nombres de establecimientos ni marcas, pero se estandariza su formato.


In [10]:
!pip install requests beautifulsoup4 pandas
!pip install requests beautifulsoup4 pandas lxml

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Codigo donde automatizo la descarga y la unificacion de los txt

In [11]:
# PASO 1: Instalar las bibliotecas necesarias
!pip install requests beautifulsoup4 pandas lxml

# PASO 2: Importar bibliotecas
import os
import requests
import zipfile
import pandas as pd
from bs4 import BeautifulSoup
from io import BytesIO
import unicodedata

# Crear carpeta para guardar los datos
os.makedirs("Datos", exist_ok=True)

# Página del SAT con los archivos de importaciones
url_base = "https://portal.sat.gob.gt/portal/alza-e-importacion-vehiculos/"

# Descargar el contenido HTML de la página
response = requests.get(url_base)
soup = BeautifulSoup(response.content, "lxml")

# Buscar todos los enlaces a archivos ZIP
zip_links = []
for link in soup.find_all("a", href=True):
    href = link["href"]
    if href.endswith(".zip"):
        if any(anio in href for anio in ["2024", "2025"]):
            if "2025" in href:
                if not any(mes in href.lower() for mes in ["01", "02", "03", "04", "05", "enero", "febrero", "marzo", "abril", "mayo"]):
                    continue  # Solo hasta mayo 2025
            zip_links.append(href)

# PASO 3: Descargar y descomprimir archivos ZIP
for zip_link in zip_links:
    zip_url = zip_link if zip_link.startswith("http") else "https://portal.sat.gob.gt" + zip_link
    zip_name = zip_url.split("/")[-1]
    print(f"⬇️ Descargando: {zip_name}")
    r = requests.get(zip_url)
    if r.status_code == 200:
        with zipfile.ZipFile(BytesIO(r.content)) as z:
            z.extractall("Datos")
            print(f"✅ Extraído: {zip_name}")
    else:
        print(f"❌ Error al descargar: {zip_name}")

# PASO 4: Leer archivos .txt descomprimidos y unir
archivos_txt = [f for f in os.listdir("Datos") if f.endswith(".txt")]
dfs = []
print("📑 Archivos .txt encontrados:", archivos_txt)


for archivo in archivos_txt:
    path = os.path.join("Datos", archivo)
    try:
        df = pd.read_csv(path, sep="|", encoding="latin1")
        df["archivo_origen"] = archivo
        dfs.append(df)
    except Exception as e:
        print(f"❌ Error leyendo {archivo}: {e}")

# PASO 5: Unir y guardar en CSV
if dfs:
    df_final = pd.concat(dfs, ignore_index=True)
    df_final.to_csv("importaciones_2024_2025.csv", index=False, encoding="utf-8-sig")
    print("📁 Archivo final generado: importaciones_2024_2025.csv")
else:
    print("⚠️ No se encontraron archivos .txt válidos.")




[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
⬇️ Descargando: alza_de_vehiculos_2025_junio.zip
✅ Extraído: alza_de_vehiculos_2025_junio.zip
⬇️ Descargando: alza_de_vehiculos_2025_junio.zip
✅ Extraído: alza_de_vehiculos_2025_junio.zip
⬇️ Descargando: alza_de_vehiculos_2025_mayo.zip
✅ Extraído: alza_de_vehiculos_2025_mayo.zip
⬇️ Descargando: alza_de_vehiculos_2025_mayo.zip
✅ Extraído: alza_de_vehiculos_2025_mayo.zip
⬇️ Descargando: alza_de_vehiculos_2025_abril.zip
✅ Extraído: alza_de_vehiculos_2025_abril.zip
⬇️ Descargando: alza_de_vehiculos_2025_abril.zip
✅ Extraído: alza_de_vehiculos_2025_abril.zip
⬇️ Descargando: alza_de_vehiculos_2025_marzo.zip
✅ Extraído: alza_de_vehiculos_2025_marzo.zip
⬇️ Descargando: alza_de_vehiculos_2025_marzo.zip
✅ Extraído: alza_de_vehiculos_2025_marzo.zip
⬇️ Descargando: alza_de_vehiculos_2025_febrero.zip
✅ Extraído: alza_de_vehiculos_2025_febrero.zip
⬇️ Descargando: alza_de_vehiculos_2025_febrero.zip
✅ Extraído: alza_de_vehic

## 1. Descripción del dataset (filas, columnas, nombres)



In [12]:
import pandas as pd

# Cargar el archivo
df = pd.read_csv("importaciones_2024_2025.csv", encoding="utf-8-sig")

# Mostrar dimensiones
print(f"➡️ Filas: {df.shape[0]}")
print(f"➡️ Variables: {df.shape[1]}\n")

# Mostrar nombres de columnas
print("📋 Nombres de columnas:")
print(df.columns.tolist())


➡️ Filas: 968871
➡️ Variables: 16

📋 Nombres de columnas:
['Pais de Proveniencia', 'Aduana de Ingreso', ' Fecha de la Poliza', 'Partida Arancelaria', 'Modelo del Vehiculo', 'Marca', 'Linea', 'Centimetros Cubicos', 'Asientos', 'Puertas', 'Tonelaje', 'Distintivo', 'Tipo de Vehiculo', 'Tipo de Importador', 'Tipo Combustible', 'archivo_origen']


## 2. Identificar columnas que necesitan limpieza

In [13]:
# Inspección general de las primeras filas
df.head()

# Detectar columnas con valores nulos
print("\n🔍 Columnas con valores faltantes:")
print(df.isnull().sum())

# Analizar tipos de datos para detectar errores
print("\n📌 Tipos de datos detectados:")
print(df.dtypes)

# Columnas de texto que suelen requerir limpieza
columnas_texto = df.select_dtypes(include='object').columns.tolist()
print("\n✏️ Columnas de texto candidatas para limpieza:")
print(columnas_texto)

# Renombrar columnas a snake_case y sin tildes
df.columns = (
    df.columns
      .str.strip()
      .map(lambda x: unicodedata.normalize('NFKD', x))
      .map(lambda x: ''.join(c for c in x if unicodedata.category(c) != 'Mn'))
      .str.lower()
      .str.replace(r"\s+", "_", regex=True)
      .str.replace(r"[^\w_]", "", regex=True)
)

# Eliminar la columna que no limpias: 'archivo_origen'
if 'archivo_origen' in df.columns:
    df.drop('archivo_origen', axis=1, inplace=True)


🔍 Columnas con valores faltantes:
Pais de Proveniencia        0
Aduana de Ingreso           0
 Fecha de la Poliza         0
Partida Arancelaria         0
Modelo del Vehiculo         0
Marca                       0
Linea                       4
Centimetros Cubicos      1152
Asientos                 1040
Puertas                  1041
Tonelaje                    2
Distintivo              14415
Tipo de Vehiculo            0
Tipo de Importador          0
Tipo Combustible            0
archivo_origen              0
dtype: int64

📌 Tipos de datos detectados:
Pais de Proveniencia     object
Aduana de Ingreso        object
 Fecha de la Poliza      object
Partida Arancelaria       int64
Modelo del Vehiculo       int64
Marca                    object
Linea                    object
Centimetros Cubicos     float64
Asientos                float64
Puertas                 float64
Tonelaje                float64
Distintivo               object
Tipo de Vehiculo         object
Tipo de Importador       o

In [14]:
# Limpiar nombres de columnas (quitar espacios, minúsculas, etc.)
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# Ver vista previa
print("🧾 Primeros registros:")
display(df.head())

# Verificar valores nulos
print("\n🔍 Valores nulos por variable:")
print(df.isnull().sum())

# Revisar valores únicos en columnas clave de texto
columnas_relevantes = ['pais_de_proveniencia', 'aduana_de_ingreso', 'marca', 'linea', 'modelo_del_vehiculo', 'distintivo']

for col in columnas_relevantes:
    print(f"\n🔎 Valores únicos en '{col}':")
    print(df[col].dropna().unique()[:10])  # primeros 10 valores


🧾 Primeros registros:


,pais_de_proveniencia,aduana_de_ingreso,fecha_de_la_poliza,partida_arancelaria,modelo_del_vehiculo,marca,linea,centimetros_cubicos,asientos,puertas,tonelaje,distintivo,tipo_de_vehiculo,tipo_de_importador,tipo_combustible
0,INDIA,PUERTO QUETZAL,06/11/2024,8711209000,2025,BAJAJ,PULSAR 125,124.0,2.0,0.0,0.0,LIVIANO,MOTO,OCASIONAL,Gasolina
1,INDIA,PUERTO QUETZAL,06/11/2024,8711209000,2025,BAJAJ,PULSAR 125,124.0,2.0,0.0,0.0,LIVIANO,MOTO,OCASIONAL,Gasolina
2,INDIA,PUERTO QUETZAL,06/11/2024,8711209000,2025,BAJAJ,PULSAR 125,124.0,2.0,0.0,0.0,LIVIANO,MOTO,OCASIONAL,Gasolina
3,INDIA,PUERTO QUETZAL,06/11/2024,8711209000,2025,BAJAJ,PULSAR 125,124.0,2.0,0.0,0.0,LIVIANO,MOTO,OCASIONAL,Gasolina
4,INDIA,PUERTO QUETZAL,06/11/2024,8711209000,2025,BAJAJ,PULSAR 125,124.0,2.0,0.0,0.0,LIVIANO,MOTO,OCASIONAL,Gasolina



🔍 Valores nulos por variable:
pais_de_proveniencia        0
aduana_de_ingreso           0
fecha_de_la_poliza          0
partida_arancelaria         0
modelo_del_vehiculo         0
marca                       0
linea                       4
centimetros_cubicos      1152
asientos                 1040
puertas                  1041
tonelaje                    2
distintivo              14415
tipo_de_vehiculo            0
tipo_de_importador          0
tipo_combustible            0
dtype: int64

🔎 Valores únicos en 'pais_de_proveniencia':
['INDIA' 'CHINA' 'JAPON' 'ESTADOS UNIDOS' 'MEXICO' 'CANADA' 'INDONESIA'
 'ARGENTINA' 'BRASIL' 'ALEMANIA REP. FED.']

🔎 Valores únicos en 'aduana_de_ingreso':
['PUERTO QUETZAL' 'EL CARMEN' 'G8' 'SANTO TOMAS DE CASTILLA' 'G1'
 'CENTRAL DE GUATEMALA' 'ADMINISTRADORA LOGISTICA DE PACIFICO,S.A'
 'TECUN UMAN' 'PUERTO BARRIOS' 'ADUANA DE VEHICULOS']

🔎 Valores únicos en 'marca':
['BAJAJ' 'YAMAHA' 'TOYOTA' 'TMR' 'MAZDA' 'SUBARU' 'VOLKSWAGEN' 'BMW' 'JAC'
 'PORSCHE']

## 4. Limpieza básica de nombres de columnas

In [15]:
# Estandarizar nombres de columnas para facilitar el análisis
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
print("\n✅ Nombres de columnas estandarizados:")
print(df.columns.tolist())



✅ Nombres de columnas estandarizados:
['pais_de_proveniencia', 'aduana_de_ingreso', 'fecha_de_la_poliza', 'partida_arancelaria', 'modelo_del_vehiculo', 'marca', 'linea', 'centimetros_cubicos', 'asientos', 'puertas', 'tonelaje', 'distintivo', 'tipo_de_vehiculo', 'tipo_de_importador', 'tipo_combustible']


##  5. Limpieza de variables de texto

In [ ]:
# Convertir columnas de texto a mayúsculas y quitar espacios extra
columnas_a_normalizar = ['pais_de_proveniencia', 'aduana_de_ingreso', 'marca', 'linea',
                         'distintivo', 'tipo_de_vehiculo', 'tipo_importador', 'tipo_combustible']

for col in columnas_a_normalizar:
    if col in df.columns:
        df[col] = df[col].astype(str).str.upper().str.strip()

print("\n🧹 Columnas normalizadas:")
print(columnas_a_normalizar)



🧹 Columnas normalizadas:
['pais_de_proveniencia', 'aduana_de_ingreso', 'marca', 'linea', 'distintivo', 'tipo_de_vehiculo', 'tipo_importador', 'tipo_combustible']


In [23]:
import pandas as pd
import numpy as np

# ✳️ Normalización de texto en columnas clave
df['pais_de_proveniencia'] = df['pais_de_proveniencia'].astype(str).str.strip().str.upper()
df['aduana_de_ingreso'] = df['aduana_de_ingreso'].astype(str).str.strip().str.upper()
df['marca'] = df['marca'].astype(str).str.strip().str.upper()
df['linea'] = df['linea'].astype(str).str.strip()
df['modelo_del_vehiculo'] = df['modelo_del_vehiculo'].astype(str).str.strip()
df['distintivo'] = df['distintivo'].astype(str).str.strip().str.upper()
df['tipo_combustible'] = df['tipo_combustible'].astype(str).str.strip().str.capitalize()

# ✳️ Convertir la fecha de la póliza a formato datetime
df['fecha_poliza'] = pd.to_datetime(df['fecha_de_la_poliza'], format="%d/%m/%Y", errors='coerce')

# ✳️ Extraer año y mes
df['anio'] = df['fecha_poliza'].dt.year
df['mes'] = df['fecha_poliza'].dt.month

# ✳️ Eliminar duplicados (opcional si hay repetidos exactos)
df.drop_duplicates(inplace=True)

print("✅ Limpieza preliminar realizada.")

variables_texto = [
    'pais_de_proveniencia', 'aduana_de_ingreso',
    'modelo_del_vehiculo', 'marca', 'linea',
    'distintivo', 'tipo_de_vehiculo',
    'tipo_de_importador', 'tipo_combustible'
]
variables_fecha = ['fecha_de_la_poliza']
variables_num   = [
    'partida_arancelaria', 'centimetros_cubicos',
    'asientos', 'puertas', 'tonelaje'
]

def clean_text(series: pd.Series, to_upper=True, strip_spaces=True) -> pd.Series:
    s = series.astype(str)
    if strip_spaces:
        s = s.str.strip()
    if to_upper:
        s = s.str.upper()
    return s.replace({'': np.nan})

def clean_numeric(series: pd.Series) -> pd.Series:
    s = series.astype(str)
    s = s.str.replace(r'\.', '', regex=True)
    s = s.str.replace(',', '.', regex=False)
    return pd.to_numeric(s, errors='coerce')

✅ Limpieza preliminar realizada.


# 6. Corrección manual de errores ortográficos


In [24]:

for var in variables_texto:
    clean_col = f"{var}_clean"
    df[clean_col] = clean_text(df[var])
    unique_vals = df[clean_col].dropna().unique()
    print(f"Valores únicos en '{clean_col}' (muestra):", sorted(unique_vals)[:20])
    print("... total unique:", len(unique_vals))

for var in variables_fecha:
    df[f"{var}_clean"] = pd.to_datetime(df[var], dayfirst=True, errors='coerce')

for var in variables_num:
    df[f"{var}_clean"] = clean_numeric(df[var])

df_clean_cols = [f"{v}_clean" for v in variables_texto + variables_fecha + variables_num]
print(df[df_clean_cols].info())
print(df[[f"{v}_clean" for v in variables_texto]].head(10))

Valores únicos en 'pais_de_proveniencia_clean' (muestra): ['ALEMANIA REP. FED.', 'ARABIA SAUDITA', 'ARGENTINA', 'AUSTRALIA', 'AUSTRIA', 'BELGICA', 'BELICE', 'BOUVET ISLAND', 'BRASIL', 'BRITISH INDIAN OCEAN TERRITORY', 'CANADA', 'CHECOSLOVAQUIA', 'CHILE', 'CHINA', 'COLOMBIA', 'COREA DEL SUR', 'COSTA RICA', 'ECUADOR', 'EMIRATOS ARABES UNIDOS', 'ESPANA']
... total unique: 64
Valores únicos en 'aduana_de_ingreso_clean' (muestra): ['ADMINISTRADORA LOGISTICA DE PACIFICO,S.A', 'ADUANA DE VEHICULOS', 'ADUANA NO REQUERIDA', 'CENTRAL DE GUATEMALA', 'CORRAL BLANCO,SOCIEDAD ANONIMA', 'DIRECCION GENERAL DE ADUANAS', 'EL CARMEN', 'EL CEIBO', 'EXPRESS AEREO', 'G1', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'H6', 'H7', 'H8', 'LA ERMITA']
... total unique: 29
Valores únicos en 'modelo_del_vehiculo_clean' (muestra): ['1900', '1928', '1930', '1935', '1936', '1939', '1942', '1944', '1947', '1948', '1949', '1950', '1951', '1952', '1953', '1954', '1956', '1957', '1959', '1960']
... total unique: 86
Valores únicos

## 7. Guardar dataset limpio

In [ ]:
# Guardar archivo limpio
df.to_csv("importaciones_limpio.csv", index=False, encoding="utf-8-sig")
print("\n💾 Dataset limpio guardado como 'importaciones_limpio.csv'")



💾 Dataset limpio guardado como 'importaciones_limpio.csv'


Aquí tienes una descripción del conjunto de datos y una estrategia de limpieza para las variables que podrían necesitar más atención:

**Descripción del Conjunto de Datos:**

*   **Filas (datos crudos):** Este número representa la cantidad total de registros de importación de vehículos en el período cubierto por los archivos descargados.
*   **Columnas (variables):** Este número indica la cantidad de atributos o características registradas para cada importación de vehículo.

**Variables que necesitarán más operaciones de limpieza:**

Basándonos en la experiencia común con este tipo de datos y los nombres de las columnas, las siguientes variables podrían requerir operaciones de limpieza:

1.  **`aduana_de_ingreso`**: A menudo contiene variaciones en la escritura del mismo nombre de aduana.
2.  **`marca`**: Es común encontrar diferentes formas de escribir la misma marca (ej: "Toyota", "TOYOTA", "Toyota ").
3.  **`linea`**: Similar a la marca, las líneas o modelos pueden tener inconsistencias en la escritura.
4.  **`modelo_del_vehiculo`**: Los años del modelo pueden presentarse en diferentes formatos o con errores.
5.  **`distintivo`**: Aunque ya se realizó una normalización inicial, podría haber otras variaciones o valores que necesiten ser estandarizados.
6.  **`tipo_de_vehiculo`**: Podría haber sinónimos o errores tipográficos.
7.  **`tipo_de_importador`**: Similar a las anteriores, puede contener inconsistencias.
8.  **`tipo_combustible`**: Podría haber diferentes formas de nombrar el mismo tipo de combustible.

**Estrategia de Limpieza para Variables Clave:**

Aquí se detalla una estrategia para limpiar las variables identificadas, manteniendo la precisión ortográfica donde sea posible:

*   **`aduana_de_ingreso`**:
    *   Convertir todo a mayúsculas o minúsculas para uniformar.
    *   Eliminar espacios en blanco al inicio y al final.
    *   Identificar y corregir errores ortográficos comunes o variaciones en los nombres de las aduanas utilizando técnicas como *fuzzy matching* o diccionarios de corrección si es necesario.
    *   Verificar si hay nombres de aduanas que sean sinónimos y unificarlos (ej: "Puerto Quetzal" y "Pto. Quetzal").

*   **`marca`, `linea`, `modelo_del_vehiculo`**:
    *   Convertir todo a mayúsculas para estandarizar.
    *   Eliminar espacios en blanco al inicio y al final.
    *   Identificar y corregir errores ortográficos o variaciones en los nombres (ej: "TOYOTA COROLLA" vs "COROLLA TOYOTA"). Esto podría requerir un análisis más profundo y posiblemente el uso de listas de marcas y modelos conocidos.
    *   Para `modelo_del_vehiculo`, asegurar que los años del modelo estén en un formato numérico consistente.

*   **`distintivo`, `tipo_de_vehiculo`, `tipo_de_importador`, `tipo_combustible`**:
    *   Convertir todo a mayúsculas para estandarizar.
    *   Eliminar espacios en blanco al inicio y al final.
    *   Identificar los valores únicos y buscar sinónimos o errores tipográficos para unificar categorías. Esto se puede hacer manualmente revisando los valores únicos o utilizando técnicas de agrupamiento de texto.

**Consideraciones Adicionales:**

*   **Fechas (`fecha_de_la_poliza`):** Aunque ya se realizó una conversión a formato datetime, es importante verificar si hay valores nulos (`NaT`) resultantes de errores en el formato original y decidir cómo manejarlos (ej: eliminarlos, imputarlos si es posible).
*   **Valores Numéricos (`centimetros_cubicos`, `asientos`, `puertas`, `tonelaje`, `modelo_del_vehiculo` como año):** Verificar tipos de datos y convertir a numéricos si es necesario. Identificar y manejar valores atípicos o erróneos (ej: números negativos, valores irrazonables).
*   **Variables con pocos valores únicos (`pais_de_proveniencia`):** Aunque pueden requerir menos limpieza, es bueno revisar los valores únicos para asegurar que no haya inconsistencias menores.

Esta estrategia busca mejorar la calidad del conjunto de datos, haciéndolo más consistente y confiable para análisis posteriores, al tiempo que se preserva la información relevante y la ortografía correcta en los nombres de los establecimientos.